In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
series = pd.read_csv('datasets/series.csv', encoding= 'unicode_escape')
train = pd.read_csv('datasets/observations_train.csv')
test = pd.read_csv('datasets/observations_test.csv')

In [3]:
# correct datatype
train.date = pd.to_datetime(train.date)
test.date = pd.to_datetime(test.date)

In [5]:
# include sp500 which has 'Daily, Close'
daily_id = series[(series.frequency == 'Daily') | (series.frequency == 'Daily, 7-Day') |\
                  (series.frequency == 'Daily, Close')]['ï»¿series_id']
weekly_id = series[(series.frequency == 'Weekly, Ending Saturday') | (series.frequency == 'Weekly, Ending Thursday') |\
                   (series.frequency == 'Weekly, Ending Wednesday')]['ï»¿series_id']
monthly_id = series[(series.frequency == 'Monthly')]['ï»¿series_id']

In [6]:
# separate data and exclude weekends
train_daily = train[train['series_id'].isin(daily_id)]
train_weekly = train[train['series_id'].isin(weekly_id)]
train_monthly = train[train['series_id'].isin(monthly_id)]
train_daily = train_daily[train_daily['date'].dt.dayofweek < 5]

test_daily = test[test['series_id'].isin(daily_id)]
test_weekly = test[test['series_id'].isin(weekly_id)]
test_monthly = test[test['series_id'].isin(monthly_id)]
test_daily = test_daily[test_daily['date'].dt.dayofweek < 5]

In [7]:
# set index for date
for df in [train_daily, train_weekly, train_monthly, test_daily, test_weekly, test_monthly]:
    df.set_index('date', inplace=True)

In [8]:
# a total of 1 na in train ser after interpolation
train_daily = train_daily.groupby('series_id').apply(lambda group: group.interpolate('time')).dropna()
train_weekly = train_weekly.groupby('series_id').apply(lambda group: group.interpolate('time'))
train_monthly = train_monthly.groupby('series_id').apply(lambda group: group.interpolate('time'))

# a total of 0 na in train ser after interpolation
test_daily = test_daily.groupby('series_id').apply(lambda group: group.interpolate('time'))
test_weekly = test_weekly.groupby('series_id').apply(lambda group: group.interpolate('time'))
test_monthly = test_monthly.groupby('series_id').apply(lambda group: group.interpolate('time'))

In [9]:
# return normalized dataset and stats to unnormalize
def in_group_norm(df):
    stats = pd.DataFrame()
    stats['series_id'] = df.series_id.unique().tolist()
    stats['mean'] = df.groupby('series_id').mean().value.tolist()
    stats['std'] = df.groupby('series_id').std().value.tolist()
    df.value = df.groupby('series_id').transform(lambda x: (x - x.mean()) / x.std()).value
    return df, stats

In [10]:
# normalize the data
train_daily, train_daily_stats = in_group_norm(train_daily)
train_weekly, train_weekly_stats = in_group_norm(train_weekly)
train_monthly, train_monthly_stats = in_group_norm(train_monthly)

test_daily, test_daily_stats = in_group_norm(test_daily)
test_weekly, test_weekly_stats = in_group_norm(test_weekly)
test_monthly, test_monthly_stats = in_group_norm(test_monthly)

In [13]:
# concat back to train and test set
train_cleaned = pd.concat([train_daily, train_weekly, train_monthly])
test_cleaned = pd.concat([test_daily, test_weekly, test_monthly])

train_cleaned.to_csv('datasets/train_cleaned.csv')
test_cleaned.to_csv('datasets/test_cleaned.csv')